In [1]:
import sys
import os

# Добавьте путь к корню вашего проекта
project_path = "/home/andrey/PycharmProjects/ProtoLLM/"
sys.path.append(project_path)

In [2]:
import chromadb
import uuid
from dotenv import load_dotenv
from langchain_community.embeddings.huggingface_hub import HuggingFaceHubEmbeddings
from protollm_sdk.models.job_context_models import PromptModel
from protollm_sdk.jobs.outer_llm_api import OuterLLMAPI
from protollm.rags.rag_core.retriever import DocRetriever, DocsSearcherModels
from definitions import CONFIG_PATH

Функция для подключения к ChromaDB

In [3]:
def init_chroma_client():
    host, port = os.environ.get("CHROMA_DEFAULT_SETTINGS").split(':')
    return chromadb.HttpClient(
        host=host,
        port=int(port),
        settings=chromadb.Settings(),
    )

Функция для запроса к ChromaDB

In [4]:
def proto_view(
    query: str,
    collection: str,
    k: int = 1,
    embedding_function: HuggingFaceHubEmbeddings = None,
) -> list:
    # Возвращает k ближайших к запросу фрагментов
    embedding_host = os.environ.get("EMBEDDING_HOST")
    embedding_function = HuggingFaceHubEmbeddings(model=embedding_host)
    chroma_client = init_chroma_client()

    docs_searcher_models = DocsSearcherModels(embedding_model=embedding_function, chroma_client=chroma_client)
    retriever = DocRetriever(top_k=k,
                             docs_searcher_models=docs_searcher_models,
                             )

    return retriever.retrieve_top(collection_name=collection, query=query)

Функция для запроса к API VseGPT

In [5]:
def outer_llm(question: str,
              meta: dict,
              key: str,
              model: str):
    llmapi = OuterLLMAPI(key, model)
    llm_request = PromptModel(job_id=str(uuid.uuid4()),
                              meta=meta,
                              content=question)
    res = llmapi.inference(llm_request)
    return res.content

In [8]:
load_dotenv(CONFIG_PATH)

# Настройки БЯМ
model = "openai/gpt-4o-2024-08-06"
meta = {"temperature": 0.05,
        "tokens_limit": 4096,
        "stop_words": None}
key = os.environ.get("VSE_GPT_KEY")

# Название коллекции в БД
collection_name = "strategy-spb"

# Вопрос
question = 'Какие задачи Стратегия ставит в области энергосбережения?'

# Извлечение контекста из БД
context = proto_view(question, collection_name)
context = f'Вопрос: {question} \nКонтекст: {context[0].page_content}'
print(context)

Вопрос: Какие задачи Стратегия ставит в области энергосбережения? 
Контекст: В рамках указанной задачи необходимо обеспечить соблюдение единых стандартов обслуживания, ускорение и оптимизацию процессов подключения к сетям инженерной инфраструктуры: электро-, тепло-, газо-, водоснабжения и водоотведения. Регулирование процессов технологического присоединения к инженерным сетям необходимо осуществлять за счет оптимизации процессов технологического присоединения, обеспечения минимального количества административных процедур и требуемых документов как для потребителя услуги по подключению, так и для организаций инженерно-энергетического комплекса при строительстве сетей инженерного обеспечения. 3.5. Задача "Развитие энергосбережения и энергетической эффективности систем коммунальной инфраструктуры". Выполнение указанной задачи направлено на развитие энергосбережения и энергоэффективности, включающее в себя реализацию организационных, правовых, технических, технологических, экономических и 

In [10]:
# Получение ответа от БЯМ
print(f'Ответ VseGPT LLM: \n {outer_llm(context, meta, key, model)}')

Ответ VseGPT LLM: 
 Стратегия в области энергосбережения ставит следующие задачи:

1. **Развитие энергосбережения и энергоэффективности**: Это включает реализацию различных мер (организационных, правовых, технических, технологических, экономических и других), направленных на уменьшение объема используемых энергетических ресурсов при сохранении их полезного эффекта.

2. **Развитие альтернативных источников энергии**: Поощрение использования и развития альтернативных источников энергии для снижения зависимости от традиционных энергетических ресурсов.

Эти задачи направлены на повышение эффективности использования энергетических ресурсов и снижение их потребления, что способствует устойчивому развитию и снижению негативного воздействия на окружающую среду.
